# Pytorch Tutorial

Pytorch is a python framework for machine learning

- GPU-accelerated computations
- automatic differentiation
- modules for neural networks

This tutorial will teach you the fundamentals of operating on pytorch tensors and networks. You have already seen some things in recitation 0 which we will quickly review, but most of this tutorial is on mostly new or more advanced stuff.

For a worked example of how to build and train a pytorch network, see `pytorch-example.ipynb`.

For additional tutorials, see http://pytorch.org/tutorials/

In [4]:
import torch
import numpy as np
import torch.nn as nn

## Tensors (review)

Tensors are the fundamental object for array data. The most common types you will use are `IntTensor` and `FloatTensor`.

In [5]:
# Create uninitialized tensor
x = torch.FloatTensor(2,3)
print(x)
# Initialize to zeros
x.zero_()
print(x)

tensor([[ 0.0000e+00,  0.0000e+00,  3.3967e-17],
        [-4.6577e-10,  1.1003e+24,  1.2944e-08]])
tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])


In [6]:
# Create from numpy array (seed for repeatability)
np.random.seed(123)
np_array = np.random.random((2,3))
print(torch.FloatTensor(np_array))
print(torch.from_numpy(np_array))

tensor([[ 0.6965,  0.2861,  0.2269],
        [ 0.5513,  0.7195,  0.4231]])
tensor([[ 0.6965,  0.2861,  0.2269],
        [ 0.5513,  0.7195,  0.4231]], dtype=torch.float64)


In [7]:
# Create random tensor (seed for repeatability)
torch.manual_seed(123)
x=torch.randn(2,3)
print(x)
# export to numpy array
x_np = x.numpy()
print(x_np)

tensor([[-0.1115,  0.1204, -0.3696],
        [-0.2404, -1.1969,  0.2093]])
[[-0.11146712  0.12036294 -0.3696345 ]
 [-0.24041797 -1.1969243   0.20926936]]


In [8]:
# special tensors (see documentation)
print(torch.eye(3))
print(torch.ones(2,3))
print(torch.zeros(2,3))
print(torch.arange(0,3))

tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]])
tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.]])
tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])
tensor([ 0.,  1.,  2.])


All tensors have a `size` and `type`

In [9]:
x=torch.FloatTensor(3,4)
print(x.size())
print(x.type())

torch.Size([3, 4])
torch.FloatTensor


## Math, Linear Algebra, and Indexing (review)

Pytorch math and linear algebra is similar to numpy. Operators are overridden so you can use standard math operators (`+`,`-`, etc.) and expect a tensor as a result. See pytorch documentation for a complete list of available functions.

In [10]:
x = torch.arange(0,5)
print(torch.sum(x))
print(torch.sum(torch.exp(x)))
print(torch.mean(x))

tensor(10.)
tensor(85.7910)
tensor(2.)


Pytorch indexing is similar to numpy indexing. See pytorch documentation for details.

In [11]:
x = torch.rand(3,2)
print(x)
print(x[1,:])

tensor([[ 0.0756,  0.1966],
        [ 0.3164,  0.4017],
        [ 0.1186,  0.8274]])
tensor([ 0.3164,  0.4017])


## CPU and GPU

Tensors can be copied between CPU and GPU. It is important that everything involved in a calculation is on the same device. 

This portion of the tutorial may not work for you if you do not have a GPU available.

In [ ]:
# create a tensor
x = torch.rand(3,2)
print(x)
# copy to GPU
y = x.cuda()
print(y)
# copy back to CPU
z = y.cpu()
print(z)
# get CPU tensor as numpy array
print(z.numpy())
# cannot get GPU tensor as numpy array directly
try:
  y.numpy()
except RuntimeError as e:
  print(e)

Operations between GPU and CPU tensors will fail. Operations require all arguments to be on the same device.

In [33]:
x = torch.rand(3,5)  # CPU tensor
y = torch.rand(5,4).cuda()  # GPU tensor
try:
  torch.mm(x,y)  # Operation between CPU and GPU fails
except TypeError as e:
  print(e)

torch.mm received an invalid combination of arguments - got (torch.FloatTensor, torch.cuda.FloatTensor), but expected one of:
 * (torch.FloatTensor source, torch.FloatTensor mat2)
      didn't match because some of the arguments have invalid types: (torch.FloatTensor, torch.cuda.FloatTensor)
 * (torch.SparseFloatTensor source, torch.FloatTensor mat2)
      didn't match because some of the arguments have invalid types: (torch.FloatTensor, torch.cuda.FloatTensor)



Typical code should include `if` statements or utilize helper functions so it can operate with or without the GPU.

In [34]:
# Put tensor on CUDA if available
x = torch.rand(3,2)
if torch.cuda.is_available():
  x = x.cuda()

# Do some calculations
y = x ** 2 

# Copy to CPU if on GPU
if y.is_cuda:
  y = y.cpu()

A convenient method is `new`, which creates a new tensor on the same device as another tensor. It should be used for creating tensors whenever possible.

In [35]:
x1 = torch.rand(3,2)
x2 = x1.new(1,2)  # create cpu tensor
print(x2)
x1 = torch.rand(3,2).cuda()
x2 = x1.new(1,2)  # create cuda tensor
print(x2)


 1.4406e+06  4.5734e-41
[torch.FloatTensor of size 1x2]


 0.1280  0.5219
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]



Calculations executed on the GPU can be many times faster than numpy. However, numpy is still optimized for the CPU and many times faster than python `for` loops. Numpy calculations may be faster than GPU calculations for small arrays due to the cost of interfacing with the GPU.

In [36]:
from timeit import timeit
# Create random data
x = torch.rand(1000,64)
y = torch.rand(64,32)
number = 10000  # number of iterations

def square():
  z=torch.mm(x, y) # dot product (mm=matrix multiplication)

# Time CPU
print('CPU: {}ms'.format(timeit(square, number=number)*1000))
# Time GPU
x, y = x.cuda(), y.cuda()
print('GPU: {}ms'.format(timeit(square, number=number)*1000))

CPU: 185.08557113818824ms
GPU: 52.751455921679735ms


## Differentiation

Tensors provide automatic differentiation.

As you might know, previous versions of Pytorch used Variables, which were wrappers around tensors for differentiation. Starting with pytorch 0.4.0, this wrapping is done internally in the Tensor class and you can, and should, differentiate Tensors directly. However, it is possible that you walk on references to Variables, e.g. in your error messages.

What you need to remember :

- Tensors you are differentiating with respect to must have `requires_grad=True`
- Call `.backward()` on scalar variables you are differentiating
- To differentiate a vector, sum it first

In [12]:
# Create differentiable tensor
x = torch.tensor(torch.arange(0,4), requires_grad=True)
# Calculate y=sum(x**2)
y = x**2
# Calculate gradient (dy/dx=2x)
y.sum().backward()
# Print values
print(x)
print(y)
print(x.grad)

tensor([ 0.,  1.,  2.,  3.])
tensor([ 0.,  1.,  4.,  9.])
tensor([ 0.,  2.,  4.,  6.])


Differentiation accumulates gradients. This is sometimes what you want and sometimes not. **Make sure to zero gradients between batches if performing gradient descent or you will get strange results!**

In [13]:
# Create a variable
x=torch.tensor(torch.arange(0,4), requires_grad=True)
# Differentiate
torch.sum(x**2).backward()
print(x.grad)
# Differentiate again (accumulates gradient)
torch.sum(x**2).backward()
print(x.grad)
# Zero gradient before differentiating
x.grad.data.zero_()
torch.sum(x**2).backward()
print(x.grad)

tensor([ 0.,  2.,  4.,  6.])
tensor([  0.,   4.,   8.,  12.])
tensor([ 0.,  2.,  4.,  6.])


Note that a Tensor with gradient cannot be exported to numpy directly :

In [14]:
x=torch.tensor(torch.arange(0,4), requires_grad=True)
x.numpy() # raises an exception

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

The reason is that pytorch remembers the graph of all computations to perform differenciation. To be integrated to this graph the raw data is wrapped internally to the Tensor class (like what was formerly a Variable). You can detach the tensor from the graph using the **.detach()** method, which returns a tensor with the same data but requires_grad set to False.

In [15]:
x=torch.tensor(torch.arange(0,4), requires_grad=True)
y=x**2
z=y**2
z.detach().numpy()

array([ 0.,  1., 16., 81.], dtype=float32)

Another reason to use this method is that updating the graph can use a lot of memory. If you are in a context where you have a differentiable tensor that you don't need to differentiate, think of detaching it from the graph.

## Neural Network Modules

Pytorch provides a framework for developing neural network modules. They take care of many things, the main one being wrapping and tracking a list of parameters for you.
You have several ways of building and using a network, offering different tradeoffs between freedom and simplicity.

torch.nn provides basic 1-layer nets, such as Linear (perceptron) and activation layers.

In [16]:
x = torch.arange(0,32)
net = torch.nn.Linear(32,10)
y = net(x)
print(y)

tensor([ 11.5822,  -1.9862,  14.1805,   5.3723,   1.9367,  -5.8406,
        -11.5825,  14.0956,  12.5262,  -7.0499])


All nn.Module objects are reusable as components of bigger networks ! That is how you build personnalized nets. The simplest way is to use the nn.Sequential class.

You can also create your own class that inherits n.Module. The forward method should precise what happens in the forward pass given an input. This enables you to precise behaviors more complicated than just applying layers one after another, if necessary.

In [17]:
# create a simple sequential network (`nn.Module` object) from layers (other `nn.Module` objects).
# Here a MLP with 2 layers and sigmoid activation.
net = torch.nn.Sequential(
    torch.nn.Linear(32,128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128,10))

In [18]:
# create a more customizable network module (equivalent here)
class MyNetwork(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size,output_size)

    def forward(self, input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h

net = MyNetwork(32,128,10)

The network tracks parameters, and you can access them through the **parameters()** method, which returns a python generator.

In [19]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0914, -0.1410, -0.1612,  ...,  0.0039, -0.1229, -0.0204],
        [ 0.0104, -0.0732,  0.0486,  ...,  0.0797,  0.0237, -0.1745],
        [ 0.0532,  0.1423, -0.0205,  ..., -0.1413,  0.0120, -0.1228],
        ...,
        [-0.0851, -0.0668,  0.0695,  ...,  0.1764,  0.0027, -0.0931],
        [-0.1475, -0.0176, -0.1509,  ...,  0.0139, -0.0923, -0.1109],
        [-0.1513, -0.1019, -0.1299,  ..., -0.1663, -0.0874,  0.0098]])
Parameter containing:
tensor([-0.1361,  0.1213, -0.1093, -0.1311, -0.1273,  0.1518, -0.0988,
        -0.0905, -0.1317,  0.0209,  0.0389,  0.1195,  0.1663, -0.0897,
        -0.1556,  0.1120,  0.1077,  0.1760,  0.1691, -0.0580, -0.0145,
        -0.1578,  0.0175, -0.0486, -0.1148, -0.0834, -0.0391,  0.0975,
        -0.0538,  0.0197,  0.0966,  0.1327, -0.0276,  0.0549, -0.0014,
         0.0823, -0.0015, -0.0788, -0.0588,  0.0901, -0.0841,  0.0586,
         0.0262,  0.1208,  0.0515, -0.0637, -0.0205,  0.1027,  0.0601,
         0.0068, -0.0869,

Parameters are of type Parameter, which is basically a wrapper for a tensor. How does pytorch retrieve your network's parameters ? They are simply all the attributes of type Parameter in your network. Moreover, if an attribute is of type nn.Module, its own parameters are added to your network's parameters ! This is why, when you define a network by adding up basic components such as nn.Linear, you should never have to explicitely define parameters.

However, if you are in a case where no pytorch default module does what you need, you can define parameters explicitely (this should be rare). For the record, let's build the previous MLP with personnalized parameters.

In [20]:
class MyNetworkWithParams(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(MyNetworkWithParams,self).__init__()
        self.layer1_weights = nn.Parameter(torch.randn(input_size,hidden_size))
        self.layer1_bias = nn.Parameter(torch.randn(hidden_size))
        self.layer2_weights = nn.Parameter(torch.randn(hidden_size,output_size))
        self.layer2_bias = nn.Parameter(torch.randn(output_size))
        
    def forward(self,x):
        h1 = torch.matmul(x,self.layer1_weights) + self.layer1_bias
        h1_act = torch.max(h1, torch.zeros(h1.size())) # ReLU
        output = torch.matmul(h1_act,self.layer2_weights) + self.layer2_bias
        return output

net = MyNetworkWithParams(32,128,10)

Parameters are useful in that they are meant to be all the network's weights that will be optimized during training. If you were needing to use a tensor in your computational graph that you want to remain constant, just define it as a regular tensor.

## Training

In [21]:
net = MyNetwork(32,128,10)

The nn.Module also provides loss functions, such as cross-entropy.

In [22]:
x = torch.tensor([np.arange(32), np.zeros(32),np.ones(32)]).float()
y = torch.tensor([0,3,9])
criterion = nn.CrossEntropyLoss()

output = net(x)
loss = criterion(output,y)
print(loss)

tensor(2.3035)


nn.CrossEntropyLoss does both the softmax and the actual cross-entropy : given $output$ of size $(n,d)$ and $y$ of size $n$ and values in $0,1,...,d-1$, it computes $\sum_{i=0}^{n-1}log(s[i,y[i]])$ where $s[i,j] = \frac{e^{output[i,j]}}{\sum_{j'=0}^{d-1}e^{output[i,j']}}$

You can also compose nn.LogSoftmax and nn.NLLLoss to get the same result. Note that all these use the log-softmax rather than the softmax, for stability in the computations.

In [23]:
# equivalent
criterion2 = nn.NLLLoss()
sf = nn.LogSoftmax()
output = net(x)
loss = criterion(sf(output),y)
loss

/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """


tensor(2.3035)

Now, to perform the backward pass, just execute **loss.backward()** ! It will update gradients in all differentiable tensors in the graph, which in particular includes all the network parameters.

In [24]:
loss.backward()

# Check that the parameters now have gradients
for param in net.parameters():
    print(param.grad)

tensor([[ 7.9256e-03,  7.6095e-03,  7.2935e-03,  ..., -1.2395e-03,
         -1.5556e-03, -1.8716e-03],
        [ 6.0576e-03,  6.0517e-03,  6.0459e-03,  ...,  5.8876e-03,
          5.8817e-03,  5.8759e-03],
        [ 4.6112e-03,  2.8569e-03,  1.1025e-03,  ..., -4.6265e-02,
         -4.8019e-02, -4.9773e-02],
        ...,
        [-5.4263e-03, -5.4263e-03, -5.4263e-03,  ..., -5.4264e-03,
         -5.4264e-03, -5.4264e-03],
        [ 5.0389e-03,  5.2486e-03,  5.4582e-03,  ...,  1.1120e-02,
          1.1329e-02,  1.1539e-02],
        [ 1.5522e-03,  1.5693e-03,  1.5865e-03,  ...,  2.0494e-03,
          2.0666e-03,  2.0837e-03]])
tensor(1.00000e-02 *
       [ 0.7206,  0.2780,  0.4303,  0.2088, -0.5509, -0.4945,  0.3033,
         0.5389, -0.5690,  0.0489,  0.5832, -0.2049, -0.1358, -0.1814,
         0.6309,  0.1223,  0.1294,  0.2739,  0.0308,  0.0416,  0.2922,
         0.7409, -0.8522, -1.0807, -0.0079,  0.5953,  0.2272,  0.5640,
        -0.1492, -0.7269, -0.5388, -0.3805,  0.8224,  0.4618,  

In [25]:
# if I forward prop and backward prop again, gradients accumulate :
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

# you can remove this behavior by reinitializing the gradients in your network parameters :
net.zero_grad()
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

tensor([[ 0.0159,  0.0152,  0.0146,  ..., -0.0025, -0.0031, -0.0037],
        [ 0.0121,  0.0121,  0.0121,  ...,  0.0118,  0.0118,  0.0118],
        [ 0.0092,  0.0057,  0.0022,  ..., -0.0925, -0.0960, -0.0995],
        ...,
        [-0.0109, -0.0109, -0.0109,  ..., -0.0109, -0.0109, -0.0109],
        [ 0.0101,  0.0105,  0.0109,  ...,  0.0222,  0.0227,  0.0231],
        [ 0.0031,  0.0031,  0.0032,  ...,  0.0041,  0.0041,  0.0042]])
tensor(1.00000e-02 *
       [ 1.4413,  0.5560,  0.8606,  0.4177, -1.1018, -0.9890,  0.6065,
         1.0778, -1.1381,  0.0977,  1.1665, -0.4099, -0.2717, -0.3628,
         1.2618,  0.2446,  0.2588,  0.5479,  0.0616,  0.0832,  0.5843,
         1.4819, -1.7043, -2.1615, -0.0159,  1.1907,  0.4545,  1.1279,
        -0.2985, -1.4539, -1.0777, -0.7609,  1.6447,  0.9236,  0.8502,
        -0.1219, -0.4525, -1.3703,  0.0223,  0.7700,  0.6401,  1.7022,
         0.0055, -0.6738, -0.8629, -0.8833,  0.3604, -1.4954, -0.0141,
         0.1449, -0.2100, -0.6163, -0.2020, -0.7

We did backpropagation, but still didn't perform gradient descent. Let's define an optimizer on the network parameters.

In [26]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

print("Parameters before gradient descent :")
for param in net.parameters():
    print(param)

optimizer.step()

print("Parameters after gradient descent :")
for param in net.parameters():
    print(param)

Parameters before gradient descent :
Parameter containing:
tensor([[-0.1337,  0.1232,  0.0619,  ...,  0.0630,  0.0150,  0.0703],
        [ 0.1393,  0.1379,  0.0322,  ..., -0.1695, -0.0602,  0.0817],
        [-0.0893, -0.0196,  0.0742,  ..., -0.0124,  0.1540, -0.1503],
        ...,
        [-0.1230, -0.0295, -0.0187,  ...,  0.1406, -0.0945, -0.1394],
        [ 0.0768, -0.1488, -0.0586,  ..., -0.1046, -0.0873,  0.0184],
        [ 0.1067,  0.1486,  0.1531,  ..., -0.1580,  0.0494,  0.1125]])
Parameter containing:
tensor([-0.0457, -0.0032,  0.0048, -0.1488,  0.0679,  0.0036, -0.1065,
         0.1504,  0.0788, -0.0569, -0.0598, -0.1160, -0.0674, -0.0856,
        -0.1280, -0.0335, -0.1202, -0.0329,  0.1357, -0.0265,  0.0350,
         0.1000, -0.0399,  0.0109, -0.1117, -0.0145,  0.0957,  0.0403,
         0.0655,  0.0713,  0.1729,  0.1131, -0.1663,  0.0978,  0.0194,
         0.0700,  0.0766, -0.0120,  0.1037, -0.0971, -0.1386,  0.0835,
         0.1103, -0.0894,  0.0634, -0.1210, -0.0439, -0.042

In [27]:
# In a training loop, we should perform many GD iterations.
n_iter = 1000
for i in range(n_iter):
    optimizer.zero_grad() # equivalent to net.zero_grad()
    output = net(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(2.1645)
tensor(2.0416)
tensor(1.9329)
tensor(1.8363)
tensor(1.7503)
tensor(1.6735)
tensor(1.6045)
tensor(1.5419)
tensor(1.4850)
tensor(1.4329)
tensor(1.3851)
tensor(1.3412)
tensor(1.3008)
tensor(1.2635)
tensor(1.2288)
tensor(1.1966)
tensor(1.1666)
tensor(1.1386)
tensor(1.1123)
tensor(1.0877)
tensor(1.0646)
tensor(1.0428)
tensor(1.0222)
tensor(1.0028)
tensor(0.9845)
tensor(0.9671)
tensor(0.9506)
tensor(0.9349)
tensor(0.9200)
tensor(0.9057)
tensor(0.8922)
tensor(0.8792)
tensor(0.8668)
tensor(0.8550)
tensor(0.8436)
tensor(0.8328)
tensor(0.8223)
tensor(0.8123)
tensor(0.8026)
tensor(0.7933)
tensor(0.7843)
tensor(0.7757)
tensor(0.7673)
tensor(0.7593)
tensor(0.7515)
tensor(0.7439)
tensor(0.7366)
tensor(0.7295)
tensor(0.7227)
tensor(0.7160)
tensor(0.7095)
tensor(0.7032)
tensor(0.6971)
tensor(0.6911)
tensor(0.6853)
tensor(0.6796)
tensor(0.6741)
tensor(0.6687)
tensor(0.6635)
tensor(0.6584)
tensor(0.6534)
tensor(0.6485)
tensor(0.6437)
tensor(0.6390)
tensor(0.6344)
tensor(0.6300)
tensor(0.6

tensor(0.1272)
tensor(0.1270)
tensor(0.1267)
tensor(0.1264)
tensor(0.1262)
tensor(0.1259)
tensor(0.1257)
tensor(0.1254)
tensor(0.1251)
tensor(0.1249)
tensor(0.1246)
tensor(0.1244)
tensor(0.1241)
tensor(0.1239)
tensor(0.1236)
tensor(0.1234)
tensor(0.1231)
tensor(0.1229)
tensor(0.1226)
tensor(0.1224)
tensor(0.1221)
tensor(0.1219)
tensor(0.1216)
tensor(0.1214)
tensor(0.1211)
tensor(0.1209)
tensor(0.1207)
tensor(0.1204)
tensor(0.1202)
tensor(0.1199)
tensor(0.1197)
tensor(0.1195)
tensor(0.1192)
tensor(0.1190)
tensor(0.1188)
tensor(0.1185)
tensor(0.1183)
tensor(0.1180)
tensor(0.1178)
tensor(0.1176)
tensor(0.1173)
tensor(0.1171)
tensor(0.1169)
tensor(0.1167)
tensor(0.1164)
tensor(0.1162)
tensor(0.1160)
tensor(0.1157)
tensor(0.1155)
tensor(0.1153)
tensor(0.1151)
tensor(0.1148)
tensor(0.1146)
tensor(0.1144)
tensor(0.1142)
tensor(0.1140)
tensor(0.1137)
tensor(0.1135)
tensor(0.1133)
tensor(0.1131)
tensor(0.1129)
tensor(0.1126)
tensor(0.1124)
tensor(0.1122)
tensor(0.1120)
tensor(0.1118)
tensor(0.1

tensor(1.00000e-02 *
       7.4449)
tensor(1.00000e-02 *
       7.4338)
tensor(1.00000e-02 *
       7.4228)
tensor(1.00000e-02 *
       7.4117)
tensor(1.00000e-02 *
       7.4007)
tensor(1.00000e-02 *
       7.3898)
tensor(1.00000e-02 *
       7.3788)
tensor(1.00000e-02 *
       7.3679)
tensor(1.00000e-02 *
       7.3570)
tensor(1.00000e-02 *
       7.3461)
tensor(1.00000e-02 *
       7.3353)
tensor(1.00000e-02 *
       7.3245)
tensor(1.00000e-02 *
       7.3137)
tensor(1.00000e-02 *
       7.3029)
tensor(1.00000e-02 *
       7.2922)
tensor(1.00000e-02 *
       7.2815)
tensor(1.00000e-02 *
       7.2708)
tensor(1.00000e-02 *
       7.2601)
tensor(1.00000e-02 *
       7.2495)
tensor(1.00000e-02 *
       7.2389)
tensor(1.00000e-02 *
       7.2283)
tensor(1.00000e-02 *
       7.2178)
tensor(1.00000e-02 *
       7.2072)
tensor(1.00000e-02 *
       7.1967)
tensor(1.00000e-02 *
       7.1863)
tensor(1.00000e-02 *
       7.1758)
tensor(1.00000e-02 *
       7.1654)
tensor(1.00000e-02 *
       

In [28]:
output = net(x)
print(output)
print(y)

tensor([[ 7.9596, -1.4903, -1.5973,  0.2635, -1.4419, -1.2406, -1.1636,
         -1.2870, -1.6135,  4.0923],
        [ 0.4499, -1.1107, -1.1447,  6.3220, -1.2572, -1.0299, -1.3596,
         -1.2432, -1.0662,  3.7187],
        [ 2.1048, -1.2875, -1.2229,  3.2101, -1.2439, -1.1290, -1.3430,
         -1.3652, -1.3110,  5.8874]])
tensor([ 0,  3,  9])


Now you know how to train a network ! For a complete training check the pytorch_example notebook.

## Saving and Loading

In [29]:
# get dictionary of keys to weights using `state_dict`
net = torch.nn.Sequential(
    torch.nn.Linear(28*28,256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256,10))
print(net.state_dict().keys())

odict_keys(['0.weight', '0.bias', '2.weight', '2.bias'])


In [30]:
# save a dictionary
torch.save(net.state_dict(),'test.t7')
# load a dictionary
net.load_state_dict(torch.load('test.t7'))

## Common issues to look out for

### Type mismatch

In [33]:
net = nn.Linear(4,2)
x = torch.tensor([1,2,3,4])
y = net(x)
print(y)

RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #2 'mat2'

In [35]:
x = x.float()
x = torch.tensor([1.,2.,3.,4.])

In [43]:
x = 2* torch.ones(2,2)
y = 3* torch.ones(2,2)
print(x * y)
print(x.matmul(y))

tensor([[ 6.,  6.],
        [ 6.,  6.]])
tensor([[ 12.,  12.],
        [ 12.,  12.]])


In [47]:
x = torch.ones(4,5)
y = torch.arange(5)
print(x+y)
y = torch.arange(4).view(-1,1)
print(x+y)
y = torch.arange(4)
print(x+y) # exception

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 1.,  2.,  3.,  4.,  5.],
        [ 1.,  2.,  3.,  4.,  5.],
        [ 1.,  2.,  3.,  4.,  5.]])
tensor([[ 1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.]])


RuntimeError: The size of tensor a (5) must match the size of tensor b (4) at non-singleton dimension 1

In [48]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)
print(x.t())
print(x.view(3,2))

tensor([[ 1,  2,  3],
        [ 4,  5,  6]])
tensor([[ 1,  4],
        [ 2,  5],
        [ 3,  6]])
tensor([[ 1,  2],
        [ 3,  4],
        [ 5,  6]])


In [ ]:
net = nn.Sequential(nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,120))
x = torch.ones(256,2048)
y = torch.zeros(256).long()
net.cuda()
x.cuda()
crit=nn.CrossEntropyLoss()
out = net(x)
loss = crit(out,y)
loss.backward()

In [56]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
    
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out

In [57]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
        self.hidden = nn.ModuleList(self.hidden)
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out